# Pristine Zr GB properties

In [1]:
from pprint import pprint

import numpy as np

from utilities import (
    load_structures,
    get_all_interface_energies, 
    get_all_wsep,
    get_interplanar_spacing_data,
    get_local_volume_change_data,
    get_coordination_change_data,
    get_structural_parameter_data,
    get_predicted_GB_widths,
    plot_GB_energies,
    plot_FS_energies,
    plot_works_of_separation,
    plot_vol_change,
    plot_coordination,
    plot_interplanar_spacing_data,    
)

In [2]:
DFT_sims = load_structures(method='DFT', json_path='data/processed/DFT_sims.json')
EAM_sims = load_structures(method='EAM', json_path='data/processed/EAM_sims.json')

## Interface energetics

In [3]:
# DFT predictions:
E_GB_DFT = get_all_interface_energies(DFT_sims, 'gb')
E_FS_DFT = get_all_interface_energies(DFT_sims, 'fs')

W_sep_DFT = get_all_wsep(DFT_sims)

# Get pristine W_GB and W_B:
W_GB_DFT = {k: v for k, v in W_sep_DFT.items() if len(k.split('-')) == 3 and '-gb' in k}
W_B_DFT = {k: v for k, v in W_sep_DFT.items() if len(k.split('-')) == 3 and '-b' in k}

# EAM E_GB predictions:
E_GB_EAM = get_all_interface_energies(EAM_sims, 'gb')

### Grain boundary energy

In [4]:
plot_GB_energies(DFT_sims, E_GB_DFT)

FigureWidget({
    'data': [{'name': 's13-tlA-gb',
              'type': 'scatter',
              'uid': '23ad…

### Surface energy

In [5]:
plot_FS_energies(DFT_sims, E_FS_DFT)

FigureWidget({
    'data': [{'name': 's13-tlA-fs',
              'type': 'scatter',
              'uid': '9826…

### Work of separation - GB

In [6]:
plot_works_of_separation(DFT_sims, W_GB_DFT)

FigureWidget({
    'data': [{'name': 's13-tlA-gb',
              'type': 'scatter',
              'uid': '1581…

### Work of separation - Bulk

In [7]:
plot_works_of_separation(DFT_sims, W_B_DFT)

FigureWidget({
    'data': [{'name': 's13-tlA-b',
              'type': 'scatter',
              'uid': '83cc1…

### Compare EAM- with DFT-predicted grain boundary energies

For EAM-DFT comparison, we compare grain boundary energies as predicted using the same methodology with both methods, i.e. from the minimum-energy γ-surface translation as predicted by the EAM or DFT methods, instead of the same γ-surface translation for both methods.

In [9]:
E_GB_DFT['s7-tlA-gb'] = E_GB_DFT.pop('s7-tlAMin-gb')

In [12]:
from utilities import write_manuscript_figure_E_GB_DFT_vs_EAM

fig = write_manuscript_figure_E_GB_DFT_vs_EAM(E_GB_EAM, E_GB_DFT, latex_labels=True)

## Geometric measures

In [4]:
struct_params = get_structural_parameter_data()
lat_a_DFT, lat_c_DFT = struct_params['lattice_parameters']['dft'].values()
lat_a_EAM, lat_c_EAM = struct_params['lattice_parameters']['eam'].values()

vol_bulk_DFT = lat_a_DFT ** 2 * np.cos(np.deg2rad(30)) * lat_c_DFT / 2
vol_bulk_EAM = lat_a_EAM ** 2 * np.cos(np.deg2rad(30)) * lat_c_EAM / 2

### Interplanar spacing

In [5]:
params = [
    {'structure_code': 's7-tlA-gb', 'step': 2, 'add_one': False, 'bulk_val': 1.05779},
    {'structure_code': 's7-tlB-gb', 'step': 2, 'add_one': False, 'bulk_val': 0.610715},
    {'structure_code': 's13-tlA-gb', 'step': 2, 'add_one': False, 'bulk_val': 0.7762077},
    {'structure_code': 's19-tlA-gb', 'step': 2, 'add_one': False, 'bulk_val': 0.642054},
    {'structure_code': 's31-tlA-gb', 'step': 2, 'add_one': False, 'bulk_val': 0.502652},
    {'structure_code': 's7-tw-gb', 'step': 1, 'add_one': True, 'bulk_val': lat_c_DFT/2, 'average_by': 7},
    {'structure_code': 's13-tw-gb', 'step': 1, 'add_one': True, 'bulk_val': lat_c_DFT/2, 'average_by': 13},
    {'structure_code': 's19-tw-gb', 'step': 1, 'add_one': True, 'bulk_val': lat_c_DFT/2, 'average_by': 19},
]

int_spacing_data_DFT = [get_interplanar_spacing_data(DFT_sims, **i) for i in params]
int_spacing_data_EAM = [get_interplanar_spacing_data(EAM_sims, **i) for i in params]

In [6]:
plot_interplanar_spacing_data(int_spacing_data_DFT)

FigureWidget({
    'data': [{'name': 's7-tlA-gb',
              'type': 'scatter',
              'uid': 'a171c…

In [7]:
plot_interplanar_spacing_data(int_spacing_data_EAM)

FigureWidget({
    'data': [{'name': 's7-tlA-gb',
              'type': 'scatter',
              'uid': '03a22…

#### Manuscript figures

In [8]:
from utilities import write_manuscript_figures_interplanar_spacing

figs = write_manuscript_figures_interplanar_spacing(DFT_sims, int_spacing_data_DFT, int_spacing_data_EAM)

### Change in local atomic volume

In [13]:
structure_codes = (
    's7-tlA-gb',
    's7-tlB-gb',
    's13-tlA-gb',
    's19-tlA-gb',
    's31-tlA-gb',
    's7-tw-gb',
    's13-tw-gb',
    's19-tw-gb',
)
vol_change_DFT = [
    get_local_volume_change_data(DFT_sims, structure_code=i, vol_bulk=vol_bulk_DFT)
    for i in structure_codes
]
vol_change_EAM = [
    get_local_volume_change_data(EAM_sims, structure_code=i, vol_bulk=vol_bulk_EAM)
    for i in structure_codes
]

In [14]:
plot_vol_change(vol_change_DFT)

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 's7-tlA-gb',
              'type': 'scat…

In [15]:
plot_vol_change(vol_change_EAM)

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 's7-tlA-gb',
              'type': 'scat…

#### Predicted GB widths

In [16]:
GB_widths_DFT, GB_widths_EAM = get_predicted_GB_widths(DFT_sims, vol_change_DFT, vol_change_EAM)

print('GB widths, lower and upper limits (DFT):')
pprint(GB_widths_DFT)

print('\nGB widths, lower and upper limits (EAM):')
pprint(GB_widths_DFT)

GB widths, lower and upper limits (DFT):
{'s13-tlA-gb': (10.491781577179648, -5.831516709577194, 4.660264867602453),
 's13-tw-gb': (5.282555138078662, -2.6412771090479286, 2.641278029030733),
 's19-tlA-gb': (8.571178126749555, -4.840081486488835, 3.73109664026072),
 's19-tw-gb': (5.343729943119532, -2.6718646457205377, 2.6718652973989947),
 's31-tlA-gb': (12.999541402634225, -7.122484416435892, 5.877056986198333),
 's7-tlA-gb': (6.501663270824777, -3.2222565862520156, 3.279406684572761),
 's7-tlB-gb': (16.22062093699097, -7.314801890064857, 8.90581904692611),
 's7-tw-gb': (5.290046712410183, -2.645024271476373, 2.6450224409338094)}

GB widths, lower and upper limits (EAM):
{'s13-tlA-gb': (10.491781577179648, -5.831516709577194, 4.660264867602453),
 's13-tw-gb': (5.282555138078662, -2.6412771090479286, 2.641278029030733),
 's19-tlA-gb': (8.571178126749555, -4.840081486488835, 3.73109664026072),
 's19-tw-gb': (5.343729943119532, -2.6718646457205377, 2.6718652973989947),
 's31-tlA-gb': (1

#### Manuscript figures

In [17]:
from utilities import write_manuscript_figures_volume_change

figs = write_manuscript_figures_volume_change(DFT_sims, vol_change_DFT, vol_change_EAM)

### Coordination

In [18]:
AREA_THRESHOLD = 1
structure_codes = (
    's7-tlA-gb',
    's7-tlB-gb',
    's13-tlA-gb',
    's19-tlA-gb',
    's31-tlA-gb',
    's7-tw-gb', 
    's13-tw-gb',
    's19-tw-gb',
)
coord_change_dat_DFT = [
    get_coordination_change_data(DFT_sims, structure_code=i, area_threshold=AREA_THRESHOLD)
    for i in structure_codes
]
coord_change_dat_EAM = [
    get_coordination_change_data(EAM_sims, structure_code=i, area_threshold=AREA_THRESHOLD)
    for i in structure_codes
]

In [19]:
plot_coordination(coord_change_dat_DFT)

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 's7-tlA-gb',
              'type': 'scat…

In [20]:
plot_coordination(coord_change_dat_EAM)

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 's7-tlA-gb',
              'type': 'scat…

#### Manuscript figures

In [21]:
from utilities import write_manuscript_figures_coordination

figs = write_manuscript_figures_coordination(DFT_sims, coord_change_dat_DFT, coord_change_dat_EAM)